In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from pylab import *
from scipy.optimize import fsolve
    
#---------------------------------------------

In [ ]:
class Setting:
    
    def __init__(self):  
        self.Points = np.zeros([0,3]) 
        #
        # x, y, type:
        # 0 - left bottom corner, 1 - left side, 2 - left top corner, 
        # 3 - top, 4 - right top corner, 5 - right side, 6 - right bottom corner, 7 - bottom,
        # 8 - normal middle, 9 - cross 
        #
        self.Edges = np.zeros([0,3]) 
        #
        # i, j, type: (always i<j on plane)
        # 0 - no edge
        # 1 - from normal go right to normal, 2 - from normal go up to normal, 
        # 3 - from normal go right and up to cross, 4 - from cross go right and up to normal,
        # 5 - from normal go right and down to cross, 6 - from cross go right and down to normal
        #
        self.BorderEdgesD = 0
        self.BorderEdgesN = 0
        self.BorderEdgesC = 0
        self.Height = 0
        self.Length = 0
        self.SizeH = 0
        self.SizeL = 0
        self.longTriangleSide = 0
        self.halfLongTriangleSide = 0
        self.shortTriangleSide = 0
        self.halfShortTriangleSide = 0
        self.TriangleArea = 0
        
            
    def indNumber(self):
        return len(self.Points) - self.BorderEdgesD - 1    


    def addPoint(self, x, y, t):       
        i=0
        while(i<len(self.Points)):
            if(self.Points[i][0] == x and self.Points[i][1] == y):
                return
            else:
                i+=1
        self.Points = np.append([[x,y,t]], self.Points, axis=0)
        for i in range (0, len(self.Edges)):
            self.Edges[i][0] += 1
            self.Edges[i][1] += 1
    
    def getPoint(self, x, y):
        i=0
        while(i<len(self.Points)):
            if(self.Points[i][0] == x and self.Points[i][1] == y):
                return i
            else:
                i+=1
        return -1
    
    def addEdge(self, i, j, t): # zawsze(i,j) ma i<j na x lub x równe oraz i<j na y 
        a = i
        b = j
        if(self.Points[j][0] < self.Points[i][0] or 
           (self.Points[j][0] == self.Points[i][0] and self.Points[j][1] < self.Points[i][1])):
            a = j
            b = i
        self.Edges = np.append([[a,b,t]], self.Edges, axis=0)
    
    
    def getEdgeType(self, i, j): #kolejność argumentów ma znaczenie
        for e in self.Edges:                
            if(e[0] == i and e[1] == j):
                return e[2]
        return 0

        
    def startBorder(self, x, y):
        self.addPoint(x,y,0)
        
    def addBorderD(self, x, y):
        self.addPoint(x,y,1)
        self.addEdge(1, 0, 2)
        self.BorderEdgesD += 1
        
    def addBorderDLast(self, x, y):
        self.addPoint(x,y,2)
        self.addEdge(1, 0, 2)
        self.BorderEdgesD += 1
        
    def addBorderNTop(self, x, y):
        self.addPoint(x,y,3)
        self.addEdge(1, 0, 1)
        self.BorderEdgesN += 1 
        
    def addBorderNTopLast(self, x, y):
        self.addPoint(x,y,4)
        self.addEdge(1, 0, 1)
        self.BorderEdgesN += 1 
        
    def addBorderNSide(self, x, y):
        self.addPoint(x,y,5)
        self.addEdge(0, 1, 2)
        self.BorderEdgesN += 1 
        
    def addBorderNSideLast(self, x, y):
        self.addPoint(x,y,6)
        self.addEdge(0, 1, 2)
        self.BorderEdgesN += 1 
        
    def addBorderC(self, x, y):
        self.addPoint(x,y,7)
        self.addEdge(0, 1, 1)
        self.BorderEdgesC += 1 
    
    def stopBorder(self):
        self.addEdge(len(self.Points)-1, 0, 1)
        self.BorderEdgesC += 1 
        
        
    def construct(self, sizeH, sizeL, height):
        self.SizeH = sizeH
        self.SizeL = sizeL
        self.Height = height
        self.longTriangleSide = float(height)/sizeH
        self.Length = self.longTriangleSide * sizeL
        
        self.halfLongTriangleSide = float(self.longTriangleSide) * 0.5
        self.shortTriangleSide = float(self.longTriangleSide) * np.sqrt(2) * 0.5
        self.halfShortTriangleSide = float(self.shortTriangleSide) * 0.5 
        self.TriangleArea = (self.longTriangleSide*self.longTriangleSide)/4.
        
        
        self.startBorder(0,0)
        
        for i in range(1,sizeH):
            self.addBorderD(0,float(i)*self.longTriangleSide)
        self.addBorderDLast(0,float(sizeH)*self.longTriangleSide)

        for i in range(1,sizeL):
            self.addBorderNTop(float(i)*self.longTriangleSide,height)
        self.addBorderNTopLast(float(sizeL)*self.longTriangleSide,height)

        for i in range(sizeH-1, 0, -1):
            self.addBorderNSide(self.Length,float(i)*self.longTriangleSide)
        self.addBorderNSideLast(self.Length,float(0))

        for i in range(sizeL-1, 0, -1):
            self.addBorderC(float(i)*self.longTriangleSide,0)
            
        self.stopBorder()

        for i in range(0,sizeL):
            for j in range(1,sizeH):
                x1 = float(i)*self.longTriangleSide
                x2 = float(i+1)*float(self.longTriangleSide)
                y = float(j)*self.longTriangleSide
                self.addPoint(x1,y,8)
                self.addPoint(x2,y,8)
                a = self.getPoint(x1,y)
                b = self.getPoint(x2,y)
                self.addEdge(a,b,1)

        for i in range(1,sizeL):
            for j in range(0,sizeH):
                x = float(i)*self.longTriangleSide
                y1 = float(j)*self.longTriangleSide
                y2 = float(j+1)*self.longTriangleSide
                self.addPoint(x,y1,8)
                self.addPoint(x,y2,8)
                a = self.getPoint(x,y1)
                b = self.getPoint(x,y2)
                self.addEdge(a,b,2)
                

        for i in range(0,sizeL):
            for j in range(0,sizeH):
                x = (float(i)+0.5)*self.longTriangleSide
                y = (float(j)+0.5)*self.longTriangleSide
                self.addPoint(x,y,9)
                a = self.getPoint(x,y)
                b = self.getPoint((float(i))*self.longTriangleSide, (float(j)+1.0)*self.longTriangleSide)
                self.addEdge(a,b,5)
                b = self.getPoint((float(i)+1.0)*self.longTriangleSide, (float(j)+1.0)*self.longTriangleSide)
                self.addEdge(a,b,4)
                b = self.getPoint((float(i)+1.0)*self.longTriangleSide, (float(j))*self.longTriangleSide)
                self.addEdge(a,b,6)
                b = self.getPoint((float(i))*self.longTriangleSide, (float(j))*self.longTriangleSide)
                self.addEdge(a,b,3)



In [ ]:
class Drawer:
    
    def __init__(self, solv):  
        self.solv = solv
        self.s = solv.s
        
        
    def draw(self):
        
        txt = 'CROSS EQUATION GR' + str(self.s.SizeH)+ ' ' + str(self.s.SizeL) \
               + ' (ml ' + str(self.solv.mi) + " "+ str(self.solv.la) \
               + ') F0[' + str(self.solv.F.F0[0]) + ',' + str(self.solv.F.F0[1]) \
               + '] FN[' + str(self.solv.F.FN[0]) + ',' + str(self.solv.F.FN[1]) + ']' 
        
        plt.close()
        axes().set_aspect('equal', 'box')
            
        shadow = 0.1  
        thickness1 = thickness2 = 2
        
        i=len(self.s.Edges)-1
        j=len(self.s.Edges)-self.s.BorderEdgesD-1
        while(j<i):
            x1 = self.s.Points[int(self.s.Edges[i,0])][0]
            y1 = self.s.Points[int(self.s.Edges[i,0])][1]
            x2 = self.s.Points[int(self.s.Edges[i,1])][0]
            y2 = self.s.Points[int(self.s.Edges[i,1])][1]
            plt.plot([x1,x2], [y1,y2], 'k-', alpha=shadow, lw=thickness1)
            i-=1
        j-=self.s.BorderEdgesN
        while(j<i):
            x1 = self.s.Points[int(self.s.Edges[i,0])][0]
            y1 = self.s.Points[int(self.s.Edges[i,0])][1]
            x2 = self.s.Points[int(self.s.Edges[i,1])][0]
            y2 = self.s.Points[int(self.s.Edges[i,1])][1]
            plt.plot([x1,x2], [y1,y2], 'k-', alpha=shadow, lw=thickness1)
            i-=1    
        j-=self.s.BorderEdgesC
        while(j<i):
            x1 = self.s.Points[int(self.s.Edges[i,0])][0]
            y1 = self.s.Points[int(self.s.Edges[i,0])][1]
            x2 = self.s.Points[int(self.s.Edges[i,1])][0]
            y2 = self.s.Points[int(self.s.Edges[i,1])][1]
            plt.plot([x1,x2], [y1,y2], 'k-', alpha=shadow, lw=thickness1)
            i-=1   
        while(-1<i):
            x1 = self.s.Points[int(self.s.Edges[i,0])][0]
            y1 = self.s.Points[int(self.s.Edges[i,0])][1]
            x2 = self.s.Points[int(self.s.Edges[i,1])][0]
            y2 = self.s.Points[int(self.s.Edges[i,1])][1]
            plt.plot([x1,x2], [y1,y2], 'k-', alpha=shadow, lw=thickness1)
            i-=1  
            
        #------------
        
       # plt.scatter(self.solv.DisplacedPoints[:,0],self.solv.DisplacedPoints[:,1], marker='o')
        
        i=len(self.s.Edges)-1
        j=len(self.s.Edges)-self.s.BorderEdgesD-1
        while(j<i):
            x1 = self.solv.DisplacedPoints[int(self.s.Edges[i,0])][0]
            y1 = self.solv.DisplacedPoints[int(self.s.Edges[i,0])][1]
            x2 = self.solv.DisplacedPoints[int(self.s.Edges[i,1])][0]
            y2 = self.solv.DisplacedPoints[int(self.s.Edges[i,1])][1]
            plt.plot([x1,x2], [y1,y2], 'r-', lw=thickness2)
            i-=1
        j-=self.s.BorderEdgesN
        while(j<i):
            x1 = self.solv.DisplacedPoints[int(self.s.Edges[i,0])][0]
            y1 = self.solv.DisplacedPoints[int(self.s.Edges[i,0])][1]
            x2 = self.solv.DisplacedPoints[int(self.s.Edges[i,1])][0]
            y2 = self.solv.DisplacedPoints[int(self.s.Edges[i,1])][1]
            plt.plot([x1,x2], [y1,y2], 'b-', lw=thickness2)
            i-=1    
        j-=self.s.BorderEdgesC
        while(j<i):
            x1 = self.solv.DisplacedPoints[int(self.s.Edges[i,0])][0]
            y1 = self.solv.DisplacedPoints[int(self.s.Edges[i,0])][1]
            x2 = self.solv.DisplacedPoints[int(self.s.Edges[i,1])][0]
            y2 = self.solv.DisplacedPoints[int(self.s.Edges[i,1])][1]
            plt.plot([x1,x2], [y1,y2], 'y-', lw=thickness2)
            i-=1   
        while(-1<i):
            x1 = self.solv.DisplacedPoints[int(self.s.Edges[i,0])][0]
            y1 = self.solv.DisplacedPoints[int(self.s.Edges[i,0])][1]
            x2 = self.solv.DisplacedPoints[int(self.s.Edges[i,1])][0]
            y2 = self.solv.DisplacedPoints[int(self.s.Edges[i,1])][1]
            plt.plot([x1,x2], [y1,y2], 'k-', lw=thickness2)
            i-=1  
            
            
        #------------
        
        plt.savefig(txt+'.jpg', transparent=True, bbox_inches='tight', pad_inches=0, dpi=300) #DPI 500
        #plt.show()
        plt.close()
        

In [ ]:
class Matrices:
    
    def __init__(self, setting, mi, la):   
        self.s = setting
        self.AX = np.zeros([self.s.indNumber(),8]) # area with dx
        self.AY = np.zeros([self.s.indNumber(),8]) # area with dy
        self.W11 = np.zeros([self.s.indNumber(),self.s.indNumber()])
        self.W12 = np.zeros([self.s.indNumber(),self.s.indNumber()])
        self.W21 = np.zeros([self.s.indNumber(),self.s.indNumber()])
        self.W22 = np.zeros([self.s.indNumber(),self.s.indNumber()])
        
        nDX = np.array([1.,1.,-1.,-1.,-1.,-1.,1.,1.]) * 0.5  #normal dx
        nDY = np.array([-1.,-1.,-1.,-1.,1.,1.,1.,1.]) * 0.5
        
        cDX = np.array([1.,0.,-1.,0.,0.,0.,0.,0.]) #cross dx
        cDY = np.array([0.,-1.,0.,1.,0.,0.,0.,0.])
        
        for i in range (0,self.s.indNumber()):
            p = self.s.Points[i]
            if(p[2] == 3): #top
                f = np.array([0,0,0,0,1,1,1,1])
                self.AX[i] = f * nDX
                self.AY[i] = f * nDY
            elif(p[2] == 4): #right top corner
                f = np.array([0,0,0,0,0,0,1,1])
                self.AX[i] = f * nDX
                self.AY[i] = f * nDY
            elif(p[2] == 5): #right
                f = np.array([1,1,0,0,0,0,1,1])
                self.AX[i] = f * nDX
                self.AY[i] = f * nDY
            elif(p[2] == 6): #right bottom corner
                f = np.array([1,1,0,0,0,0,0,0])
                self.AX[i] = f * nDX
                self.AY[i] = f * nDY
            elif(p[2] == 7): #bottom
                f = np.array([1,1,1,1,0,0,0,0])
                self.AX[i] = f * nDX
                self.AY[i] = f * nDY
            elif(p[2] == 8): #normal middle
                f = np.array([1,1,1,1,1,1,1,1])
                self.AX[i] = f * nDX
                self.AY[i] = f * nDY
            elif(p[2] == 9): #cross
                f = np.array([1,1,1,1,0,0,0,0]) # only 4 used
                self.AX[i] = f * cDX
                self.AY[i] = f * cDY
        
        self.multiply(self.W11, self.AX, self.AX)
        self.multiply(self.W12, self.AX, self.AY) 
        self.multiply(self.W21, self.AY, self.AX)#np.transpose(self.W12)
        self.multiply(self.W22, self.AY, self.AY)
        
        self.B11 = (2 * mi + la) * self.W11 + mi * self.W22
        self.B12 = mi * self.W21 + la * self.W12
        self.B21 = la * self.W21 + mi * self.W12
        self.B22 = mi * self.W11 + (2 * mi + la) * self.W22
        
        
    def multiply(self, W, AK, AL):
        for i in range (0,self.s.indNumber()):
            W[i][i] = np.sum(AK[i]*AL[i])
            
            #c - contacting triangles numbers
            for j in range (0,self.s.indNumber()):
                c1i = -1; c1j = -1; c2i = -1; c2j = -1
                
                if(self.s.getEdgeType(i,j) == 1): #1 - from normal go right to normal
                    c1i = 3; c1j = 0
                    c2i = 4; c2j = 7
                    
                elif(self.s.getEdgeType(i,j) == 2): #2 - from normal go up to normal
                    c1i = 1; c1j = 6
                    c2i = 2; c2j = 5
                    
                elif(self.s.getEdgeType(i,j) == 3): #3 - from normal go right and up to cross
                    c1i = 2; c1j = 0
                    c2i = 3; c2j = 3
                    
                elif(self.s.getEdgeType(i,j) == 4): #4 - from cross go right and up to normal
                    c1i = 1; c1j = 7
                    c2i = 2; c2j = 6
                    
                elif(self.s.getEdgeType(i,j) == 5): #5 - from normal go right and down to cross
                    c1i = 4; c1j = 1
                    c2i = 5; c2j = 0
                    
                elif(self.s.getEdgeType(i,j) == 6): #6 - from cross go right and down to normal
                    c1i = 2; c1j = 1
                    c2i = 3; c2j = 0
                
                if(c1i>=0): #edge was found
                    W[i][j] = AK[i][c1i]*AL[j][c1j]+AK[i][c2i]*AL[j][c2j]
                    W[j][i] = AL[i][c1i]*AK[j][c1j]+AL[i][c2i]*AK[j][c2j]
                    
  

In [ ]:
class F:    
    
    def __init__(self, setting, F0, FN):  
        self.F0 = F0
        self.FN = FN
        self.s = setting
        self.F = np.zeros([self.s.indNumber(),2])
        self.Zero = np.zeros([self.s.indNumber()])
        self.One = np.zeros([self.s.indNumber()])
              
            
######################################################## 
    
    def f0(self,x):
        return self.F0
    
    def fN(self,x):
        return self.FN
    
########################################################


    def setF(self):
        halfLongTriangleSide = self.s.halfLongTriangleSide
        halfShortTriangleSide = self.s.halfShortTriangleSide
        
        self.F = np.zeros([self.s.indNumber(),2])
        
        for i in range (0,self.s.indNumber()):
            x = self.s.Points[i][0]
            y = self.s.Points[i][1]
            t = self.s.Points[i][2]
            
            if(t != 9): # normal point
                
                valuesInTriangle = np.zeros([8,2])
                
                valuesInTriangle[0] += self.f0([x - halfLongTriangleSide, y])
                valuesInTriangle[0] += self.f0([x - halfShortTriangleSide, y + halfShortTriangleSide])
                    
                valuesInTriangle[1] += self.f0([x - halfShortTriangleSide, y + halfShortTriangleSide])
                valuesInTriangle[1] += self.f0([x, y + halfLongTriangleSide])
                    
                valuesInTriangle[2] += self.f0([x, y + halfLongTriangleSide])
                valuesInTriangle[2] += self.f0([x + halfShortTriangleSide, y + halfShortTriangleSide])
                    
                valuesInTriangle[3] += self.f0([x + halfShortTriangleSide, y + halfShortTriangleSide])
                valuesInTriangle[3] += self.f0([x + halfLongTriangleSide, y])
                    
                valuesInTriangle[4] += self.f0([x + halfLongTriangleSide, y])
                valuesInTriangle[4] += self.f0([x + halfShortTriangleSide, y - halfShortTriangleSide])
                    
                valuesInTriangle[5] += self.f0([x + halfShortTriangleSide, y - halfShortTriangleSide])
                valuesInTriangle[5] += self.f0([x, y - halfLongTriangleSide])
                    
                valuesInTriangle[6] += self.f0([x, y - halfLongTriangleSide])
                valuesInTriangle[6] += self.f0([x - halfShortTriangleSide, y - halfShortTriangleSide])
                    
                valuesInTriangle[7] += self.f0([x - halfShortTriangleSide, y - halfShortTriangleSide])
                valuesInTriangle[7] += self.f0([x - halfLongTriangleSide, y])
                       
                if(t == 3): # 3 - top
                    self.F[i] +=  valuesInTriangle[4]
                    self.F[i] +=  valuesInTriangle[5]
                    self.F[i] +=  valuesInTriangle[6]
                    self.F[i] +=  valuesInTriangle[7]
                if(t == 4): # 4 - right top corner
                    self.F[i] +=  valuesInTriangle[6]
                    self.F[i] +=  valuesInTriangle[7]
                if(t == 5): # 5 - right side
                    self.F[i] +=  valuesInTriangle[0]
                    self.F[i] +=  valuesInTriangle[1]
                    self.F[i] +=  valuesInTriangle[6]
                    self.F[i] +=  valuesInTriangle[7]
                if(t == 6): # 6 - right bottom corner
                    self.F[i] +=  valuesInTriangle[0]
                    self.F[i] +=  valuesInTriangle[1]
                if(t == 7): # 7 - bottom
                    self.F[i] +=  valuesInTriangle[0]
                    self.F[i] +=  valuesInTriangle[1]
                    self.F[i] +=  valuesInTriangle[2]
                    self.F[i] +=  valuesInTriangle[3]
                if(t == 8): # 8 - normal middle
                    self.F[i] +=  valuesInTriangle[0]
                    self.F[i] +=  valuesInTriangle[1]
                    self.F[i] +=  valuesInTriangle[2]
                    self.F[i] +=  valuesInTriangle[3]
                    self.F[i] +=  valuesInTriangle[4]
                    self.F[i] +=  valuesInTriangle[5]
                    self.F[i] +=  valuesInTriangle[6]
                    self.F[i] +=  valuesInTriangle[7]
                    
                self.F[i] = (float(self.s.TriangleArea)/6) * self.F[i]
                
            else: # cross point
                
                valuesInTriangle = np.zeros([4,2])
                
                valuesInTriangle[0] += self.f0([x - halfShortTriangleSide, y - halfShortTriangleSide])
                valuesInTriangle[0] += self.f0([x - halfShortTriangleSide, y + halfShortTriangleSide])
                    
                valuesInTriangle[1] += self.f0([x - halfShortTriangleSide, y + halfShortTriangleSide])
                valuesInTriangle[1] += self.f0([x + halfShortTriangleSide, y + halfShortTriangleSide])
                    
                valuesInTriangle[2] += self.f0([x + halfShortTriangleSide, y + halfShortTriangleSide])
                valuesInTriangle[2] += self.f0([x + halfShortTriangleSide, y - halfShortTriangleSide])
                    
                valuesInTriangle[3] += self.f0([x + halfShortTriangleSide, y - halfShortTriangleSide])
                valuesInTriangle[3] += self.f0([x - halfShortTriangleSide, y - halfShortTriangleSide])
                
                self.F[i] +=  valuesInTriangle[0]
                self.F[i] +=  valuesInTriangle[1]
                self.F[i] +=  valuesInTriangle[2]
                self.F[i] +=  valuesInTriangle[3]
                
                self.F[i] = (float(self.s.TriangleArea)/6) * self.F[i]
                
        for i in range (0,self.s.indNumber()):
            for e in range (-self.s.BorderEdgesD - self.s.BorderEdgesN, -self.s.BorderEdgesD):
                e1 = int(self.s.Edges[e][0])
                e2 = int(self.s.Edges[e][1])
                p1 = self.s.Points[int(e1)][0:2]
                p2 = self.s.Points[int(e2)][0:2]
                if(i == e1 or i == e2):
                    self.F[i] += ((self.s.longTriangleSide * 0.5) * self.fN((p1 + p2) * 0.5))
                    
        self.Zero = self.F[:,0]
        self.One = self.F[:,1]

In [ ]:
class Solver:

    def __init__(self, setting, timeStep, F0, FN, mi, la): 
        
        self.mi = mi
        self.la = la
        
        self.s = setting
        self.tS = timeStep
        self.currentTime = 0
        
        self.M = Matrices(setting, mi, la)
        self.F = F(setting, F0, FN)
                
        self.u = np.zeros([self.s.indNumber(),2]) 

        self.DisplacedPoints = np.zeros([len(self.s.Points),3])
        
        for i in range (0,len(self.s.Points)):
            self.DisplacedPoints[i] = self.s.Points[i]
        
        
    def norm(self, v):
        return sqrt(v[0]*v[0] + v[1]*v[1])
    
    def length(self, p1, p2):
        return float(sqrt((p1[0]-p2[0])*(p1[0]-p2[0]) + (p1[1]-p2[1])*(p1[1]-p2[1])))
    
    def nDown(self, e):
        #[0,-1]
        e1 = int(self.s.Edges[e][0])
        e2 = int(self.s.Edges[e][1])
        dx = self.s.Points[e2][0]-self.s.Points[e1][0]
        dy = self.s.Points[e2][1]-self.s.Points[e1][1]
        norm = sqrt(dx*dx + dy*dy)
        n = np.array([float(dy)/norm, float(-dx)/norm])
        if(n[1]> 0):
            n = -n
        return n
    
    def nDownAtContactBoundary(self):
        N = np.zeros([self.s.indNumber(),2])  
        
        for i in range (0,self.s.indNumber()):
            for e in range (-self.s.BorderEdgesD-self.s.BorderEdgesN -self.s.BorderEdgesC, -self.s.BorderEdgesD-self.s.BorderEdgesN):
                e1 = int(self.s.Edges[e][0])
                e2 = int(self.s.Edges[e][1])
                if(i == e1 or i == e2):
                    n = self.nDown(e)
                    if(N[i][0] == 0 and N[i][1] == 0):
                        N[i] = n
                    else:
                        N[i] = 0.5*N[i] + 0.5*n
        return N
            
             
    def JZu(self):
        JZu = np.zeros([self.s.indNumber(),2])
        
        for i in range (0,self.s.indNumber()):
            for e in range (-self.s.BorderEdgesD-self.s.BorderEdgesN -self.s.BorderEdgesC, -self.s.BorderEdgesD-self.s.BorderEdgesN):
                e1 = int(self.s.Edges[e][0])
                e2 = int(self.s.Edges[e][1])
                if(i == e1 or i == e2):
                    umL = 0 # u at mL
                    if(e1 < self.s.indNumber()):
                        umL += self.u[e1]*0.5
                    if(e2 < self.s.indNumber()):
                        umL += self.u[e2]*0.5
                        
                    p1 = self.s.Points[int(e1)][0:2]
                    p2 = self.s.Points[int(e2)][0:2]
                    mL = (p1 + p2) * 0.5
                    L = self.length(p1,p2)
                    nmL = self.nDown(e)  #n at mL
                    
                    uNmL = umL[0] * nmL[0] + umL[1] * nmL[1]
                    uTmL = umL - uNmL * nmL
                    
                    
                    vNZero = nmL[0]
                    vNOne = nmL[1]
                    vThauZero = [1. - float(nmL[0]*nmL[0]), - float(nmL[0]*nmL[1])]
                    vThauOne = [- float(nmL[0]*nmL[1]), 1. - float(nmL[1]*nmL[1])]
                    
                    JZu[i][0] += L * 0.5 * (self.jnZ(uNmL,vNZero) + self.h(uNmL) * self.jtZ(uTmL,vThauZero)) 
                    JZu[i][1] += L * 0.5 * (self.jnZ(uNmL,vNOne) + self.h(uNmL) * self.jtZ(uTmL,vThauOne)) 
        
        return JZu
    
    
    def iterate(self, uVector): 
        self.u[:,0] = uVector[0:self.s.indNumber()]
        self.u[:,1] = uVector[self.s.indNumber():2*self.s.indNumber()] 
        
        for i in range (0,self.s.indNumber()):
            self.DisplacedPoints[i][0] = self.s.Points[i][0] + self.u[i][0]
            self.DisplacedPoints[i][1] = self.s.Points[i][1] + self.u[i][1]
            
  
    def Bu1(self):
        return (np.dot(self.M.B11, self.u[:,0]) + np.dot(self.M.B12, self.u[:,1]))
    def Bu2(self):
        return (np.dot(self.M.B21, self.u[:,0]) + np.dot(self.M.B22, self.u[:,1]))
    
    def f(self, uVector):
        self.u = np.zeros([self.s.indNumber(),2]) 
        self.u[:,0] = uVector[0:self.s.indNumber()]
        self.u[:,1] = uVector[self.s.indNumber():2*self.s.indNumber()]
        
        X = self.Bu1() \
            + self.JZu()[:,0] \
            - self.F.Zero
                
        Y = self.Bu2() \
            + self.JZu()[:,1] \
            - self.F.One
        
        return 100000000 * np.append(X, Y) #10000000000
            

########################################################
    
    knu = 1.
    delta = 0.1
    
    def jnZ(self, uN, vN): #un, vN - scalars
       # return 0
        if(uN<=0):
            return 0 * vN
        return (self.knu * uN) * vN
    
    def h(self, uN):
        return 0
        #if(uN<=0):
        #    return 0
        #return 8.*uN
    
    def jtZ(self, uT, vT): #uT, vT - vectors; REGULARYZACJA Coulomba
        rho = 0.0000001
        M = 1/math.sqrt(float(uT[0]*uT[0]+uT[1]*uT[1]) + float(rho*rho))
        return M*float(uT[0])*float(vT[0])+M*float(uT[1])*float(vT[1])
        
        
########################################################
   


In [ ]:
########################################################
timeStep = 1
gridHeight = 1

gridSizeH = 2
gridSizeL = 5
F0 = np.array([-0.2,-0.2]) 
FN = np.array([0,0])
mi = 4
la = 4
########################################################


s1 = Setting()
s1.construct(gridSizeH,gridSizeL,gridHeight)
solv = Solver(s1, timeStep, F0, FN, mi, la)
d = Drawer(solv)

uVector = np.zeros([2*s1.indNumber()])
 
solv.currentTime = 1
solv.F.setF();
     
while True:
    uVector = fsolve(solv.f, uVector)
    quality = np.linalg.norm(solv.f(uVector))
    print('Q')
    print(quality)
    if quality < 1:
        break    
    
solv.iterate(uVector)
d.draw()
    
print("DONE")

In [8]:
#print(s1.Edges)
#print(s1.Points)
#print(solv.F.Zero)
#print(solv.F.One)
#print(solv.M.W12)
#print(solv.M.W11)
#print(solv.M.W22)
#print(solv.M.AX)
#print(solv.M.AY)

#print(np.array_equal(solv.M.W12, np.transpose(solv.M.W21)))
#print(np.array_equal(solv.M.W11, np.transpose(solv.M.W11)))
#print(np.array_equal(solv.M.W22, np.transpose(solv.M.W22)))